In [1]:
print("test")

test


In [2]:
%pwd

'd:\\Projects\\FinBot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\FinBot'

In [5]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
# Ensure pypdf package is installed
%pip install pypdf

extracted_data=load_pdf_file(data='Data/')

Note: you may need to restart the kernel to use updated packages.


In [10]:
#extracted_data

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2519


In [13]:
#text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
# Ensure sentence-transformers package is installed
%pip install sentence-transformers



Note: you may need to restart the kernel to use updated packages.


In [17]:
# Ensure transformers and torch packages are installed and updated
%pip install --upgrade transformers
%pip install --upgrade torch

embeddings = download_hugging_face_embeddings()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


C:\Users\deLL 10 GeN\AppData\Local\Temp\ipykernel_11144\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Installed Applications\Anaconda\envs\finBot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
#query_result

In [20]:
%pip install pinecone


Note: you may need to restart the kernel to use updated packages.


In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [23]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index details
index_name = "finbot"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd9d69be8a49bebe69b5c0008af638520', 'Date': 'Mon, 24 Feb 2025 18:12:04 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [39]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [25]:
%pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [40]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

KeyboardInterrupt: 

In [27]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [41]:
docsearch

In [42]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [43]:
retrieved_docs = retriever.invoke("What is Investment?")

In [44]:
retrieved_docs

[Document(id='ecccfb06-56e6-4abb-aea9-408ddf853d25', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2019-09-22T07:30:21+00:00', 'page': 6.0, 'page_label': '7', 'producer': 'www.ilovepdf.com', 'source': 'Data\\book.pdf', 'total_pages': 405.0}, page_content='8\nFinancial and Investment\nManagement\nan investment in an apartment for the purpose of living in it may involve, partially at least\ncertain, current consumption but because the family will continue to live in the house for\na very long period of time, the act of purchasing a house or apartment may be taken as\nan investment activity.\n1.2 MEANING OF INVESTMENT\nWe can now give a simple yet a broad definition of investment. We can define investment\nas “Postponed Consumption”.'),
 Document(id='793b1b30-9902-4347-ab75-fd576f441281', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2019-09-22T07:30:21+00:00', 'page': 6.0, 'page_label': '7', 'producer': 'www.ilovepdf.com', 'source': 'Data\\book.pdf', 'tota

In [46]:
%pip install langchain-google-genai

  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached googleapis_common_protos-1.68.0-py2.py3-none-any.whl.metadata (5.1 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.3 MB 1.2 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 1.3 MB/s eta 0:00:01
   ------------------------------- -------- 1.0/1.3 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 1.2 MB/s eta 0:00:00
Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 1.9

In [47]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.4,
    max_tokens=500,
    google_api_key=os.environ.get('GOOGLE_API_KEY')
)

In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [49]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [50]:
response = rag_chain.invoke({"input": "What is Systematic Risk"})
print(response["answer"])

Systematic risk, also known as market risk, is the portion of risk that cannot be reduced by diversifying a portfolio. It is caused by factors that affect the entire market or economy, such as interest rates, inflation, and economic growth. Systematic risk is measured by beta, which represents the volatility of a fund's NAV relative to the market.


In [52]:
response = rag_chain.invoke({"input": "You have invested Rs. 50,000, 30 percent of which is invested in Company A, which has a expected rate of return of 15 percent, and 70 percent of which is invested in Company B, which has an expected rate of return of 12 percent. What is the return on your portfolio?"})
print(response["answer"])

The total return on the portfolio is Rs. 6450. This is calculated by multiplying the amount invested in each company by its expected rate of return and then adding the results together.
